In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv(r'data/birth-rate-vs-death-rate.csv')
#Data wrangling 
continents = df[['Entity','Continent']].drop_duplicates()
continents = continents[~continents['Continent'].isna()].set_index('Entity')['Continent'].to_dict()
df['Continent'] = df['Entity'].map(continents)
df = df[~df['Continent'].isna()]
df.head()

,Entity,Code,Year,Birth rate - Sex: all - Age: all - Variant: estimates,Death rate - Sex: all - Age: all - Variant: estimates,Population - Sex: all - Age: all - Variant: estimates,Continent
0,Abkhazia,OWID_ABK,2015,NaN,NaN,NaN,Asia
1,Afghanistan,AFG,1950,48.866,37.945,7480464.0,Asia
2,Afghanistan,AFG,1951,49.147,37.329,7571542.0,Asia
3,Afghanistan,AFG,1952,49.331,36.618,7667534.0,Asia
4,Afghanistan,AFG,1953,49.541,35.995,7764549.0,Asia


In [2]:
from bokeh.plotting import figure, show, save
from bokeh.models import CustomJS, CustomJSFilter, CDSView, RangeSlider, ColumnDataSource
from bokeh.layouts import column
from bokeh.io import output_notebook, show, output_file, curdoc
from bokeh.resources import CDN
from bokeh.embed import file_html

p = figure(title='Birth rate vs Death rate of countries (in thousands)', width=1000, height=700, tooltips="@Year @Entity")
xy = p.line(x=np.arange(0,80,1), y=np.arange(0,80,1), line_alpha=0.3, legend_label='No net birth/death', line_color='white', line_dash='dashed')
p.xaxis.axis_label = 'Death rate/1000'
p.yaxis.axis_label = 'Birth rate/1000'

range_slider = RangeSlider(title='Year', start=1950, end=2021, step=1, value=(1950, 2021))

#make a massive dictionary of sources --> one for each country --> obviously cleaner ways to do this but being overly verbose/readable on purpose here
#will look like this: {'Canada':ColumnDataSource(data={'Year':[1950,1951...],'Death Rate':[x,y...]...}
                        #,'Germany':ColumnDataSource(data={...}} etc.
source_dict = {}
#hold the individual renderers in a dict too, can be good practice
rend_dict = {}
#color theme
cdict = {'Asia':'#FF3636', 'Europe':'#00A2FF', 'Africa':'#585858', 'Oceania':'#38CA00', 'North America':'#FFFF6D',
       'Antarctica':'#000000', 'South America':'#B641FE'}
#master source is a flat table of ALL country's data
master_source = ColumnDataSource(data=df)

In [3]:
#initialize source dict with all data
for c in continents.keys():
    cut = df[df['Entity']==c]
    source_dict[c] = ColumnDataSource({col:cut[col] for col in df.columns}) # Creating a CDS for each country filled with info on all the data in df.columns
    rend_dict[c] = p.line(x='Death rate - Sex: all - Age: all - Variant: estimates',y='Birth rate - Sex: all - Age: all - Variant: estimates'
                          ,source=source_dict[c],line_color=cdict[continents[c]],legend_label=continents[c])
    # Creating a CDS for each country's plotted line, setting the colour as specified in the cdict dictionary

In [4]:
cjs = CustomJS(args=dict(sl=range_slider,source_dict=source_dict
                         ,master_source=master_source),code='''
                      //range slider vals
                      var minR = sl.value[0]
                      var maxR = sl.value[1]
                      //create an "updatedd" source for all countries
                      var upd_source = {}
                      for (const [k,v] of Object.entries(source_dict)){
                              upd_source[k] = {}
                              for (const [key,val] of Object.entries(source_dict[k].data)){
                                      upd_source[k][key] = []
                                      }
                              }
                      //loop through the master source, and depending on whether it meets the range criteria, add it to
                      //the respective country's upd_source
                      for (var i = 0; i<master_source.data['Year'].length;i++){
                              var c = master_source.data['Entity'][i]
                              if (master_source.data['Year'][i]>=minR && master_source.data['Year'][i]<=maxR){
                                      for (const [k,v] of Object.entries(upd_source[c])){
                                              upd_source[c][k].push(master_source.data[k][i])
                                            }
                                  }
                            }
                      //then assign upd_source to source_dict (i.e. the source that's running the renderers')
                      for (const [k,v] of Object.entries(source_dict)){
                              v.data = upd_source[k]
                            }
                      ''')

In [5]:
range_slider.js_on_change('value',cjs)
p.legend.click_policy = 'hide'

layout = column(p, range_slider)
curdoc().theme = 'dark_minimal'

# output_notebook()

# show(layout)

html = file_html(layout, CDN)
file_path = "birthdeathrate.html"
with open(file_path, "w", encoding="utf-8") as file:
    file.write(html)